In [2]:
from importlib import reload

In [244]:
from data.eeg_datasets import inner_speech

In [245]:
inner_speech = reload(inner_speech)

In [246]:
from torchvision.transforms import v2
from data.transforms.transform import TransposeTransform, EEGFourierTransform

In [247]:
import wandb
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [248]:
transforms = v2.Compose([
    # EEGFourierTransform(),
    TransposeTransform(),
])

In [249]:
dataset = inner_speech.BIDSEEGDataset('/media/elijah/T7/ds000117-download', transform=TransposeTransform())

In [250]:
data_path = '/media/elijah/T7/ds000117-download'

## Dataset conversion

In [251]:
# schema = {'eeg': 'ndarray:float32:74,2048', 'label':'ndarray:int32:2048', 'subject':'str'}
# import os
# import mne
# import numpy as np
# import pandas as pd
# from tqdm.autonotebook import tqdm
# from sklearn.preprocessing import LabelEncoder
# from streaming import MDSWriter

# def load_file_info(bids_path):
#     file_data = []
#     for subject_dir in os.listdir(bids_path):
#         full_subject_dir = os.path.join(bids_path, subject_dir)
#         if subject_dir.startswith('sub-'):
#             for session_dir in os.listdir(full_subject_dir):
#                 if session_dir.startswith('ses-meg'):
#                     meg_dir = os.path.join(full_subject_dir, session_dir, 'meg')
#                     for file in os.listdir(meg_dir):
#                         if file.endswith('_meg.fif'):
#                             fif_path = os.path.join(meg_dir, file)
#                             events_path = fif_path.replace('_meg.fif', '_events.tsv')
#                             if os.path.exists(events_path):
#                                 file_data.append({'subject': subject_dir.split('-')[-1], 'fif_path': fif_path, 'events_path': events_path})
#     return pd.DataFrame(file_data)

# def load_subject_data(events_path, encoder):
#     events_df = pd.read_csv(events_path, sep='\t')
#     session_data = []
#     for idx, event in events_df.iterrows():
#         session_data.append({
#             'start': event['onset'],
#             'duration': event['duration'] if 'duration' in event else 1.0,  # Assuming default duration as 1.0 if not provided
#             'label': event['stim_type'],
#         })
#     df = pd.DataFrame(session_data)
#     df['label'] = encoder.transform(df['label'])
#     df['duration'] = np.diff(df['start'], append=df['start'].iloc[-1] + 1)

#     return df

# def create_label_vector(events, sfreq, start_sample, segment_length):
#     label_vector = np.zeros(segment_length, dtype=int)
#     for event in events:
#         event_start_sample = int(event['start'] * sfreq)
#         event_end_sample = event_start_sample + int(event['duration'] * sfreq)
#         event_label = event['label']
        
#         for i in range(segment_length):
#             sample_index = start_sample + i
#             if event_start_sample <= sample_index < event_end_sample:
#                 label_vector[i] = event_label
                
#     return label_vector

# def preprocess_file(file_info, encoder, fixed_length=2048, resample_rate=None, transform=None):
#     raw = mne.io.read_raw_fif(file_info['fif_path'], preload=True, verbose=False)
#     raw.pick_types(eeg=True, meg=False, verbose=False)

#     if resample_rate:
#         raw.resample(resample_rate)

#     sfreq = raw.info['sfreq']
#     total_samples = raw.n_times
#     subject = file_info['subject']

#     events = load_subject_data(file_info['events_path'], encoder).to_dict('records')
#     # yield events

#     for start in range(0, total_samples, fixed_length):
#         end = start + fixed_length
#         if end > total_samples:
#             break  # Ignore the last segment if it's smaller than the fixed length

#         eeg_segment = raw.get_data(start=start, stop=end)
#         if transform:
#             eeg_segment = transform(eeg_segment)


#         label_vector = create_label_vector(events, sfreq, start, fixed_length)

#         yield {
#             'eeg': eeg_segment.astype(np.float32),
#             'label': label_vector.astype(np.int32),
#             'subject': subject
#         }

# def fit_label_encoder(bids_path):
#     all_labels = []
#     for subject_dir in os.listdir(bids_path):
#         full_subject_dir = os.path.join(bids_path, subject_dir)
#         if subject_dir.startswith('sub-'):
#             for session_dir in os.listdir(full_subject_dir):
#                 if session_dir.startswith('ses-meg'):
#                     meg_dir = os.path.join(full_subject_dir, session_dir, 'meg')
#                     for file in os.listdir(meg_dir):
#                         if file.endswith('_meg.fif'):
#                             events_path = os.path.join(meg_dir, file.replace('_meg.fif', '_events.tsv'))
#                             if os.path.exists(events_path):
#                                 events_df = pd.read_csv(events_path, sep='\t')
#                                 all_labels.extend(events_df['stim_type'].values)
#     encoder = LabelEncoder()
#     encoder.fit(all_labels)
#     return encoder

# def data_generator(bids_path, fixed_length=2048, resample_rate=None, transform=None):
#     encoder = fit_label_encoder(bids_path)
#     print(encoder.classes_)
#     file_info_df = load_file_info(bids_path)

#     for idx in tqdm(range(len(file_info_df))):
#         file_info = file_info_df.iloc[idx]
    
#         segments = preprocess_file(file_info, encoder, fixed_length=fixed_length, resample_rate=resample_rate, transform=transform)
#         if segments:
#             for segment in segments:
#                 yield segment


# bids_path = data_path
output_path = '/media/elijah/T7/ds117_streaming_eeg/'
# with MDSWriter(out=output_path, columns=schema) as out:
#     for data in data_generator(bids_path, fixed_length=2048, resample_rate=256):
#         # print(data)
#         # if not (data['label']==data['label'][0]).all():
#         # print(data)
#         out.write(data)


In [252]:
# !rm -r /media/elijah/T7/ds117_streaming_eeg/

In [253]:
import torch
from torch.utils.data import random_split, DataLoader

## Model description

In [254]:
# self.accuracy = Accuracy(task='multiclass',num_classes=num_classes)

In [255]:
from streaming import StreamingDataset, StreamingDataLoader
batch_size = 32
dataset = StreamingDataset(local=output_path, batch_size=batch_size)
loader = StreamingDataLoader(dataset,batch_size=batch_size)

Because `predownload` was not specified, it will default to 8*batch_size if batch_size is not None, otherwise 64. Prior to Streaming v0.7.0, `predownload` defaulted to max(batch_size, 256 * batch_size // num_canonical_nodes).


In [256]:
dataset[3]

{'eeg': array([[4.5864883e-05, 2.8982113e-05, 3.1193646e-05, ..., 2.8771536e-05,
         3.1445194e-05, 3.3458284e-05],
        [3.0096837e-05, 2.9869638e-05, 2.9401632e-05, ..., 3.2504009e-05,
         3.4547022e-05, 2.8847438e-05],
        [4.5079796e-05, 4.6390494e-05, 4.2576165e-05, ..., 3.4838755e-05,
         3.2985994e-05, 2.8013463e-05],
        ...,
        [5.1696119e-05, 7.9748243e-05, 8.5277134e-05, ..., 6.6001972e-05,
         4.1906096e-05, 1.7044764e-05],
        [3.1810272e-05, 3.7320398e-05, 4.5373698e-05, ..., 4.0132742e-05,
         4.0790146e-05, 4.0565988e-05],
        [5.0761835e-05, 5.3283475e-05, 5.2247789e-05, ..., 3.6617876e-05,
         3.1128166e-05, 2.3539398e-05]], dtype=float32),
 'label': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [257]:
dataset[3]['eeg'][None, ...].T.shape

(512, 74, 1)

In [259]:
import torch
import torchaudio

def compute_spectrograms(eeg_data, fs=256, n_fft=256, hop_length=128):
    """
    Compute the spectrogram for each channel of the EEG data using PyTorch.

    Parameters:
    eeg_data (torch.Tensor): Input EEG data with shape (batch_size, sequence_length, num_channels)
    fs (int): Sampling frequency, default is 256 Hz
    n_fft (int): Number of FFT components, default is 256
    hop_length (int): Number of samples between successive frames, default is 128

    Returns:
    torch.Tensor: Spectrogram images with shape (batch_size, num_channels, freq_bins, time_bins)
    """
    batch_size, sequence_length, num_channels = eeg_data.shape
    spectrogram_images = []

    for batch_idx in range(batch_size):
        spectrograms = []
        for channel_idx in range(num_channels):
            # Compute the spectrogram using torchaudio
            channel_data = eeg_data[batch_idx, :, channel_idx]
            spec = torchaudio.transforms.Spectrogram(n_fft=n_fft, hop_length=hop_length)(channel_data)
            spectrograms.append(spec)
        spectrogram_images.append(torch.stack(spectrograms, dim=0))

    return torch.stack(spectrogram_images, dim=0)

# Example usage
eeg_data = torch.randn(10, 2048, 74)  # 10 samples, each with 2048 sequence length and 74 channels
spectrograms = compute_spectrograms(eeg_data)

print(spectrograms.shape)  # Should print (10, 74, freq_bins, time_bins)


torch.Size([10, 74, 129, 17])


In [260]:
next(iter(loader))['eeg'].permute(0, 2, 1).shape

Because `num_canonical_nodes` was not specified, and `shuffle_algo` is py1e, it will default to be equal to physical nodes. Prior to Streaming v0.7.0, `num_canonical_nodes` defaulted to 64 * physical nodes.
Because `shuffle_block_size` was not specified, it will default to max(4_000_000 // num_canonical_nodes, 1 << 18) if num_canonical_nodes is not None, otherwise 262144. Prior to Streaming v0.7.0, `shuffle_block_size` defaulted to 262144.


torch.Size([32, 512, 74])

In [261]:
next(iter(loader))['label'].shape

torch.Size([32, 512])

In [262]:
'512'.isdigit()

True

In [263]:
# import torch
# from torch import nn
# import pytorch_lightning as pl
# import torch.nn.functional as F
# import torchmetrics

# class EEGStepClassifier(pl.LightningModule):
#     def __init__(self, num_channels=74, d_model=512, num_classes=8, num_layers=3):
#         super().__init__()
#         self.save_hyperparameters()
#         self.ignore_index = -1
#         self.lstm = nn.LSTM(input_size=num_channels, hidden_size=d_model, batch_first=True, num_layers=num_layers, dropout=0.1)
#         self.embedding = nn.Embedding(num_embeddings=17, embedding_dim=d_model)
#         self.fc2 = nn.Linear(d_model * 2, d_model)  # Adjusted for concatenated embedding
#         self.gelu = nn.GELU()
#         self.mlp = nn.Sequential(
#             nn.Linear(d_model, d_model),
#             nn.GELU(),
#             nn.Linear(d_model, d_model),
#             nn.GELU(),
#         )
#         self.classifier = nn.Linear(d_model, num_classes)
        
#         self.accuracy = torchmetrics.Accuracy(task='multiclass', num_classes=num_classes)

#     def forward(self, x, id):
#         ids = torch.tensor([self.get_id(iid) for iid in id], device=self.device)
#         x, (hn, cn) = self.lstm(x)
        
#         # Get the user embeddings
#         user_emb = self.embedding(ids)
        
#         # Expand user embeddings to match the sequence length
#         # x.shape[1] provides the sequence length dynamically
#         user_emb = user_emb.unsqueeze(1).expand(-1, x.shape[1], -1)
        
#         # Concatenate along the feature dimension
#         x = torch.cat((x, user_emb), dim=-1)
        
#         x = self.gelu(self.fc2(x))
#         x = self.mlp(x)
#         logits = self.classifier(x)
#         return logits


#     def get_id(self, name: str):
#         return int(name) if name.isdigit() else 0

#     def training_step(self, batch, batch_idx):
#         x = batch['eeg'].permute(0, 2, 1)
#         y = batch['label'].view(-1).long()
#         id = batch['subject']
#         logits = self.forward(x, id)
#         loss = F.cross_entropy(logits.view(-1, logits.size(-1)), y, ignore_index=self.ignore_index)
#         mask = y != self.ignore_index
#         acc = self.accuracy(logits.view(-1, logits.size(-1))[mask], y[mask])
#         self.log('train_loss', loss, prog_bar=True, on_step=True, on_epoch=True)
#         self.log('train_acc', acc, prog_bar=True, on_step=False, on_epoch=True)
#         return loss

#     def validation_step(self, batch, batch_idx):
#         x = batch['eeg'].transpose(0, 2, 1)
#         y = batch['label'].view(-1)
#         id = batch['subject']
#         logits = self.forward(x, id)
#         loss = F.cross_entropy(logits.view(-1, logits.size(-1)), y, ignore_index=self.ignore_index)
#         mask = y != self.ignore_index
#         acc = self.accuracy(logits.view(-1, logits.size(-1))[mask], y[mask])
#         self.log('val_loss', loss, prog_bar=True, on_epoch=True)
#         self.log('val_acc', acc, prog_bar=True, on_epoch=True)

#     def configure_optimizers(self):
#         optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
#         scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3 )
#         return [optimizer], [scheduler]


In [264]:
# import torch
# from torch import nn
# import pytorch_lightning as pl
# import torch.nn.functional as F
# import torchmetrics

# class EEGTransformerClassifier(pl.LightningModule):
#     def __init__(self, num_channels=74, d_model=512, num_classes=8, num_layers=3):
#         super().__init__()
#         self.in_norm = nn.InstanceNorm1d(num_features=num_channels)
#         self.save_hyperparameters()
#         self.ignore_index = -1

#         self.transformer = nn.TransformerEncoder(
#             nn.TransformerEncoderLayer(
#                 d_model=d_model, 
#                 nhead=8, 
#                 dim_feedforward=d_model, 
#                 dropout=0.1
#             ),
#             num_layers=num_layers
#         )
#         self.fc1 = nn.Linear(num_channels, d_model)
#         self.fc2 = nn.Linear(d_model, num_classes)
#         self.accuracy = torchmetrics.Accuracy(task='multiclass', num_classes=num_classes)

#     def forward(self, x):
#         x = self.in_norm(x.permute(0, 2, 1))
#         x = self.fc1(x)  # Linearly project input to match d_model
#         x = x.permute(1, 0, 2)  # Transformer expects (seq_len, batch_size, features)
#         x = self.transformer(x)
#         x = x.permute(1, 0, 2)  # Back to (batch_size, seq_len, features)
#         logits = self.fc2(x)
#         return logits

#     def training_step(self, batch, batch_idx):
#         x, y = batch['eeg'], batch['label'].view(-1).long()
#         logits = self.forward(x)
#         loss = F.cross_entropy(logits.view(-1, self.hparams.num_classes), y, ignore_index=self.ignore_index)
#         acc = self.accuracy(logits.view(-1, self.hparams.num_classes), y)
#         self.log('train_loss', loss, prog_bar=True, on_step=True, on_epoch=True)
#         self.log('train_acc', acc, prog_bar=True, on_step=False, on_epoch=True)
#         return loss

#     def validation_step(self, batch, batch_idx):
#         x, y = batch['eeg'], batch['label'].view(-1)
#         logits = self.forward(x)
#         loss = F.cross_entropy(logits.view(-1, self.hparams.num_classes), y, ignore_index=self.ignore_index)
#         acc = self.accuracy(logits.view(-1, self.hparams.num_classes), y)
#         self.log('val_loss', loss, prog_bar=True, on_epoch=True)
#         self.log('val_acc', acc, prog_bar=True, on_epoch=True)

#     def configure_optimizers(self):
#         optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
#         # scheduler = {
#         #     'scheduler': torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3),
#         #     'monitor': 'train_loss'  # Make sure this metric is being logged in your validation_step
#         # }
#         scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.5)
#         return [optimizer], [scheduler]


In [265]:
a=1

In [266]:
import numpy as np

In [309]:
import torch
from torch import nn
import pytorch_lightning as pl
import torch.nn.functional as F
import torchmetrics
import torchaudio
from scipy.signal import butter, lfilter

class EEGStepClassifier(pl.LightningModule):
    def __init__(self, num_channels=74, d_model=512, num_classes=8, fs=256, n_fft=256, hop_length=128):
        super().__init__()
        self.save_hyperparameters()
        self.ignore_index = -1
        self.fs = fs
        self.a = 1.1
        self.n_fft = n_fft
        self.hop_length = hop_length
        self.embedding = nn.Embedding(num_embeddings=17, embedding_dim=d_model)
        
        # Convolutional layers for encoding spectrograms with residual connections
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=num_channels, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,1))
        )

        self.dropout = nn.Dropout(p=0.5)
        self.fc1 = nn.Linear(4096, d_model)
        self.mlp = nn.Sequential(
            nn.Linear(2*d_model, d_model),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(d_model, d_model),
            nn.ReLU(),
            nn.Dropout(p=0.5)
        )
        self.classifier = nn.Linear(d_model, num_classes)
        
        self.accuracy = torchmetrics.Accuracy(task='multiclass', num_classes=num_classes)

    def bandpass_filter(self, data, lowcut, highcut, fs, order=5):
        nyquist = 0.5 * fs
        low = lowcut / nyquist
        high = highcut / nyquist
        b, a = butter(order, [low, high], btype='band')
        return lfilter(b, a, data)

    def compute_spectrograms(self, eeg_data):
        batch_size, sequence_length, num_channels = eeg_data.shape
        spectrogram_images = []

        for batch_idx in range(batch_size):
            spectrograms = []
            for channel_idx in range(num_channels):
                # Compute the spectrogram using torchaudio
                channel_data = eeg_data[batch_idx, :, channel_idx]
                spec_transform = torchaudio.transforms.Spectrogram(n_fft=self.n_fft, hop_length=self.hop_length).to(eeg_data.device)
                spec = spec_transform(channel_data)
                spectrograms.append(spec)
            spectrogram_images.append(torch.stack(spectrograms, dim=0))

        return torch.stack(spectrogram_images, dim=0)

    def forward(self, x, id):
        # Band-pass filter for EEG data (e.g., focusing on 0.5-100 Hz range)
        x = torch.tensor([self.bandpass_filter(channel.cpu().numpy(), 0.5, 100, self.fs) for channel in x.permute(0, 2, 1)], device=x.device).permute(0, 2, 1).float()
        
        # Compute spectrograms
        x = self.compute_spectrograms(x)
        
        # Apply convolutional layers with residual connections
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        
        # Flatten and prepare for MLP
        batch_size, num_channels, freq_bins, time_bins = x.shape
        x = x.view(batch_size, -1)
        
        ids = torch.tensor([self.get_id(iid) for iid in id], device=self.device)
        user_emb = self.embedding(ids)
        
        x = self.dropout(self.fc1(x))
        x = torch.cat((x, user_emb), dim=-1)
        x = self.mlp(x)
        logits = self.classifier(x)
        return logits

    def get_id(self, name: str):
        return int(name) if name.isdigit() else 0

    def training_step(self, batch, batch_idx):
        x = batch['eeg'].permute(0, 2, 1)
        y = batch['label'][..., -1].long()
        id = batch['subject']
        logits = self.forward(x, id)
        loss = F.cross_entropy(logits, y, ignore_index=self.ignore_index)
        mask = y != self.ignore_index
        acc = self.accuracy(logits[mask], y[mask])
        # self.log('train_loss', loss/np.log(self.a), prog_bar=True, on_step=True, on_epoch=True)
        # self.log('train_acc', acc*np.log(self.a), prog_bar=True, on_step=False, on_epoch=True)
        # self.a+=0.002
        self.log('train_loss', loss, prog_bar=True, on_step=True, on_epoch=True)
        self.log('train_acc', acc, prog_bar=True, on_step=False, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x = batch['eeg'].permute(0, 2, 1)
        y = batch['label'].view(-1)
        id = batch['subject']
        logits = self.forward(x, id)
        loss = F.cross_entropy(logits, y, ignore_index=self.ignore_index)
        mask = y != self.ignore_index
        acc = self.accuracy(logits[mask], y[mask])
        self.log('val_loss', loss, prog_bar=True, on_epoch=True)
        self.log('val_acc', acc, prog_bar=True, on_epoch=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=3e-5)
        scheduler = {
            'scheduler': torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=3, threshold=1e-2),
            'monitor': 'train_acc'  # Make sure this metric is being logged in your validation_step
        }
        return [optimizer], [scheduler]

# Example usage (requires data loader and trainer setup)
# trainer = pl.Trainer(max_epochs=10)
# model = EEGStepClassifier()
# trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)


In [310]:
version=0

In [311]:
# n_classes = len(dataset.encoder.classes_)
n_classes = 3
# model = EEGStepClassifier(num_classes=n_classes, num_layers=2, d_model=256)
model = EEGStepClassifier(num_classes=n_classes, d_model=256)

In [312]:
# from torch.nn.utils.rnn import pad_sequence
# import torch

# def custom_collate_fn(batch):
#     x, y = zip(*batch)
#     x_padded = pad_sequence(x, batch_first=True, padding_value=0)
    
#     # Handle possible empty y values
#     if any(len(label) == 0 for label in y):
#         print("Warning: Empty label encountered!")  # Debugging line
#         y = [label if len(label) > 0 else torch.tensor([-100]) for label in y]  # Example fallback
    
#     y_padded = pad_sequence(y, batch_first=True, padding_value=-100)
#     return x_padded, y_padded

# # When creating your DataLoader, specify the custom collate function
# # dataloader = DataLoader(dataset, batch_size=64, collate_fn=custom_collate_fn, pin_memory=True, shuffle=True, num_workers=11)
# train_set_size = int(len(dataset) * 0.8)
# valid_set_size = len(dataset) - train_set_size

# # split the train set into two
# seed = torch.Generator().manual_seed(42)

# train_set, valid_set = random_split(dataset, [train_set_size, valid_set_size], generator=seed)

# train_loader = DataLoader(train_set, batch_size=16, shuffle=True, pin_memory=True, num_workers=11, collate_fn=custom_collate_fn)
# test_loader = DataLoader(valid_set, batch_size=16, shuffle=True, pin_memory=True, num_workers=11, collate_fn=custom_collate_fn)


In [313]:
# wandb.finish()

In [314]:
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger, WandbLogger
import os
version+=1
# default logger used by trainer (if tensorboard is installed)
# logger = TensorBoardLogger(save_dir=os.getcwd(), version=version, name="eeg_classification")
logger = WandbLogger(project='diploma', name='diploma_eeg_spectr_conv',)
# Initialize a trainer
trainer = Trainer(max_epochs=50, log_every_n_steps=1, logger=logger, )  # Adjust max_epochs as needed, set gpus=0 for CPU

# Train the model
trainer.fit(model, loader)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type               | Params
--------------------------------------------------
0 | embedding  | Embedding          | 4.4 K 
1 | conv1      | Sequential         | 42.8 K
2 | conv2      | Sequential         | 74.1 K
3 | conv3      | Sequential         | 295 K 
4 | dropout    | Dropout            | 0     
5 | fc1        | Linear             | 1.0 M 
6 | mlp        | Sequential         | 197 K 
7 | classifier | Linear             | 771   
8 | accuracy   | MulticlassAccuracy | 0     
--------------------------------------------------
1.7 M     Trainable params
0         Non-trainable params
1.7 M     Total params
6.655     Total estimated model params size (MB)


Training: |                                                                                                   …

/home/elijah/.cache/pypoetry/virtualenvs/eeg-mri-analyzer-C0pNHHSC-py3.10/lib/python3.10/site-packages/pytorch_lightning/utilities/data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 17. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
`Trainer.fit` stopped: `max_epochs=50` reached.


In [49]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [50]:
!nvidia-smi

Wed May 22 21:10:06 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.147.05   Driver Version: 525.147.05   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0  On |                  N/A |
|  0%   57C    P2    51W / 220W |   1402MiB /  8192MiB |      4%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!kill -9 538114